# BC 5
## Flat ensemble of tiny networks
## Input = sub-region

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import cifar10
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score

In [2]:
NUM_TN = 30
SUB_REGION_SCALE = 14

#dataset specific parameters
NUM_CLASS = 10

## Data Pre-Processing

In [3]:
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

In [5]:
#Generate cropped train image
X_train_cropped_list = []
y_train_cropped_list = []

for i in range(NUM_TN):
  X_train_cropped = []
  y_train_cropped = []
  for i in range (X_train.shape[0]):
    image_cropped = tf.image.random_crop(X_train[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
    image_cropped = np.array(image_cropped)
    X_train_cropped.append(image_cropped)
    y_train_cropped.append(y_train[i])
  X_train_cropped = np.array(X_train_cropped)
  y_train_cropped = np.array(y_train_cropped)

  X_train_cropped_list.append(X_train_cropped)
  y_train_cropped_list.append(y_train_cropped)

In [6]:
#Generate cropped test image
X_test_cropped_list = []
y_test_cropped_list = []

for i in range(NUM_TN):
  X_test_cropped = []
  y_test_cropped = []
  for i in range (X_test.shape[0]):
    image_cropped = tf.image.random_crop(X_test[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
    image_cropped = np.array(image_cropped)
    X_test_cropped.append(image_cropped)
    y_test_cropped.append(y_test[i])
  X_test_cropped = np.array(X_test_cropped)
  y_test_cropped = np.array(y_test_cropped)

  X_test_cropped_list.append(X_test_cropped)
  y_test_cropped_list.append(y_test_cropped)

In [7]:
#normalizing dataset
for i in range(NUM_TN):
  X_train_cropped_list[i] = X_train_cropped_list[i]/255
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [8]:
#transform to vector
for i in range(NUM_TN):
  X_train_cropped_list[i] = X_train_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

In [9]:
#one-hot encoding
for i in range(NUM_TN):
  y_train_cropped_list[i] = to_categorical(y_train_cropped_list[i])
  y_test_cropped_list[i] = to_categorical(y_test_cropped_list[i])

## Model

In [10]:
#build ANN model
ensemble = []
for i in range(NUM_TN):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*3))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append(model)

In [11]:
#train model
history = []
for i in range(NUM_TN):
  print('Model %d' %i)
  hist = ensemble[i].fit(X_train_cropped_list[i], y_train_cropped_list[i], validation_data=(X_test_cropped_list[i], y_test_cropped_list[i]), epochs=5, batch_size=128)
  history.append(hist)

Model 0
Epoch 1/5
391/391 [==============================] - 2s 4ms/step - loss: 2.1755 - accuracy: 0.1829 - val_loss: 2.0552 - val_accuracy: 0.2193
Epoch 2/5
391/391 [==============================] - 1s 2ms/step - loss: 2.0450 - accuracy: 0.2278 - val_loss: 2.0231 - val_accuracy: 0.2350
Epoch 3/5
391/391 [==============================] - 1s 2ms/step - loss: 2.0091 - accuracy: 0.2420 - val_loss: 2.0191 - val_accuracy: 0.2304
Epoch 4/5
391/391 [==============================] - 1s 2ms/step - loss: 1.9928 - accuracy: 0.2492 - val_loss: 1.9830 - val_accuracy: 0.2548
Epoch 5/5
391/391 [==============================] - 1s 2ms/step - loss: 1.9788 - accuracy: 0.2573 - val_loss: 1.9677 - val_accuracy: 0.2652
Model 1
Epoch 1/5
391/391 [==============================] - 1s 3ms/step - loss: 2.1828 - accuracy: 0.1724 - val_loss: 2.0658 - val_accuracy: 0.2291
Epoch 2/5
391/391 [==============================] - 1s 3ms/step - loss: 2.0465 - accuracy: 0.2321 - val_loss: 2.0335 - val_accuracy: 0.23

## Model Evaluation

In [12]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN):
  y_pred = ensemble[i].predict(X_test_cropped_list[i])
  y_pred_list.append(y_pred)
    
# Ensemble voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.argmax(np.sum(y_pred_list, axis=0), axis=1)

In [13]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.3404